# SQL: Data Cleaning and Analysis

In [6]:
import sqlite3
import pandas as pd

df = pd.read_csv("../data/wdi_edu_filtered_long.csv")
conn = sqlite3.connect("wdi_edu.db")
df.to_sql("wdi_edu", conn, if_exists="replace", index=False)

1512

In [7]:
pd.read_sql('''
SELECT *
FROM wdi_edu
''', conn)

,region,series,year,value
0,East Asia & Pacific,primary,2000,110.224030
1,East Asia & Pacific,primary,2001,110.923576
2,East Asia & Pacific,primary,2002,109.465172
3,East Asia & Pacific,primary,2003,108.066650
4,East Asia & Pacific,primary,2004,106.849770
...,...,...,...,...
1507,World,tertiary_male,2016,35.047852
1508,World,tertiary_male,2017,35.308880
1509,World,tertiary_male,2018,35.517368
1510,World,tertiary_male,2019,36.082550


## Regional Averages and Distribution of Enrollment (2020-2021)

In [ ]:
pd.read_sql('''
SELECT region, series, AVG(value) AS avg_rate
FROM wdi_edu
WHERE series IN ('primary', 'secondary', 'tertiary')
GROUP BY region, series;
''', conn)

,region,series,avg_rate
0,East Asia & Pacific,primary,104.536424
1,East Asia & Pacific,secondary,79.326682
2,East Asia & Pacific,tertiary,32.326086
3,Europe & Central Asia,primary,101.987955
4,Europe & Central Asia,secondary,100.229224
5,Europe & Central Asia,tertiary,61.597396
6,Latin America & Caribbean,primary,110.814036
7,Latin America & Caribbean,secondary,89.990020
8,Latin America & Caribbean,tertiary,40.691914
9,"Middle East, North Africa, Afghanistan & Pakistan",primary,89.024141


In [ ]:
pd.read_sql('''
SELECT 
    series,
    AVG(value)  AS mean_rate,
    MIN(value)  AS min_rate,
    MAX(value)  AS max_rate
FROM wdi_edu
WHERE series IN ('primary', 'secondary', 'tertiary')
GROUP BY series;
''', conn)

,series,mean_rate,min_rate,max_rate
0,primary,101.106546,79.473808,118.426079
1,secondary,73.880423,25.435600,105.108917
2,tertiary,36.725496,4.384480,86.950279


## Avg Enrollment for Each Education Level per Region

In [9]:
pd.read_sql('''
WITH growth AS (
    SELECT
        region,
        series,
        AVG(annual_growth) AS avg_growth
    FROM (
        SELECT
            region,
            series,
            value - LAG(value) OVER (PARTITION BY region, series ORDER BY year)
                AS annual_growth
        FROM wdi_edu
        WHERE year BETWEEN 2000 AND 2020
          AND series IN ('primary','secondary','tertiary')
    ) g
    WHERE annual_growth IS NOT NULL
    GROUP BY region, series
)
SELECT
    region,
    MAX(CASE WHEN series = 'primary' THEN avg_growth END) AS primary_growth,
    MAX(CASE WHEN series = 'secondary' THEN avg_growth END) AS secondary_growth,
    MAX(CASE WHEN series = 'tertiary' THEN avg_growth END) AS tertiary_growth
FROM growth
GROUP BY region
ORDER BY region;
''', conn)

,region,primary_growth,secondary_growth,tertiary_growth
0,East Asia & Pacific,-0.508122,1.387767,1.804751
1,Europe & Central Asia,-0.166644,0.397700,1.491138
2,Latin America & Caribbean,-0.489661,0.886855,1.552549
3,"Middle East, North Africa, Afghanistan & Pakistan",0.528657,0.525619,0.737428
4,North America,-0.031169,0.326427,0.971567
5,South Asia,0.233246,1.474988,0.949892
6,Sub-Saharan Africa,0.921688,0.963458,0.240672
7,World,0.025631,0.851748,1.011157


## Gap from World Average

In [ ]:
pd.read_sql('''
WITH regional_primary AS (
    SELECT
        region,
        AVG(value) AS avg_primary_rate
    FROM wdi_edu
    WHERE series = 'primary'
      AND region <> 'World'
    GROUP BY region
),
world_primary AS (
    SELECT
        AVG(value) AS world_primary_rate
    FROM wdi_edu
    WHERE series = 'primary'
      AND region = 'World'
)
SELECT
    r.region,
    r.avg_primary_rate,
    w.world_primary_rate,
    (w.world_primary_rate - r.avg_primary_rate) AS gap_from_world
FROM regional_primary r
CROSS JOIN world_primary w;
''', conn)

,region,avg_primary_rate,world_primary_rate,gap_from_world
0,East Asia & Pacific,104.536424,101.293432,-3.242991
1,Europe & Central Asia,101.987955,101.293432,-0.694522
2,Latin America & Caribbean,110.814036,101.293432,-9.520604
3,"Middle East, North Africa, Afghanistan & Pakistan",89.024141,101.293432,12.269291
4,North America,100.895668,101.293432,0.397765
5,South Asia,108.019121,101.293432,-6.725689
6,Sub-Saharan Africa,92.281592,101.293432,9.011840


In [43]:
pd.read_sql('''
WITH regional_secondary AS (
    SELECT
        region,
        AVG(value) AS avg_secondary_rate
    FROM wdi_edu
    WHERE series = 'secondary'
      AND region <> 'World'
    GROUP BY region
),
world_secondary AS (
    SELECT
        AVG(value) AS world_secondary_rate
    FROM wdi_edu
    WHERE series = 'secondary'
      AND region = 'World'
)
SELECT
    r.region,
    r.avg_secondary_rate,
    w.world_secondary_rate,
    (w.world_secondary_rate - r.avg_secondary_rate) AS gap_from_world
FROM regional_secondary r
CROSS JOIN world_secondary w;
''', conn)

,region,avg_secondary_rate,world_secondary_rate,gap_from_world
0,East Asia & Pacific,79.326682,69.485886,-9.840796
1,Europe & Central Asia,100.229224,69.485886,-30.743338
2,Latin America & Caribbean,89.990020,69.485886,-20.504134
3,"Middle East, North Africa, Afghanistan & Pakistan",55.052961,69.485886,14.432925
4,North America,97.163060,69.485886,-27.677174
5,South Asia,62.982683,69.485886,6.503202
6,Sub-Saharan Africa,36.812869,69.485886,32.673017


In [44]:
pd.read_sql('''
WITH regional_tertiary AS (
    SELECT
        region,
        AVG(value) AS avg_tertiary_rate
    FROM wdi_edu
    WHERE series = 'tertiary'
      AND region <> 'World'
    GROUP BY region
),
world_tertiary AS (
    SELECT
        AVG(value) AS world_tertiary_rate
    FROM wdi_edu
    WHERE series = 'tertiary'
      AND region = 'World'
)
SELECT
    r.region,
    r.avg_tertiary_rate,
    w.world_tertiary_rate,
    (w.world_tertiary_rate - r.avg_tertiary_rate) AS gap_from_world
FROM regional_tertiary r
CROSS JOIN world_tertiary w;
''', conn)

,region,avg_tertiary_rate,world_tertiary_rate,gap_from_world
0,East Asia & Pacific,32.326086,29.954535,-2.371551
1,Europe & Central Asia,61.597396,29.954535,-31.642861
2,Latin America & Caribbean,40.691914,29.954535,-10.737379
3,"Middle East, North Africa, Afghanistan & Pakistan",21.230170,29.954535,8.724365
4,North America,82.450066,29.954535,-52.495531
5,South Asia,18.264229,29.954535,11.690307
6,Sub-Saharan Africa,7.289573,29.954535,22.664962


## Gap with Europe & Central Asia + North America (most developed regions)

In [ ]:
pd.read_sql('''
WITH regional_primary AS (
    SELECT
        region,
        AVG(value) AS avg_primary_rate
    FROM wdi_edu
    WHERE series = 'primary'
      AND region NOT IN ('World', 'Europe & Central Asia', 'North America')
    GROUP BY region
),
benchmark AS (
    SELECT
        AVG(value) AS benchmark_rate
    FROM wdi_edu
    WHERE series = 'primary'
      AND region IN ('Europe & Central Asia', 'North America')
)
SELECT
    r.region,
    r.avg_primary_rate,
    b.benchmark_rate,
    (b.benchmark_rate - r.avg_primary_rate) AS gap_from_benchmark
FROM regional_primary r
CROSS JOIN benchmark b;
''', conn)


,region,avg_primary_rate,benchmark_rate,gap_from_benchmark
0,East Asia & Pacific,104.536424,101.441811,-3.094613
1,Latin America & Caribbean,110.814036,101.441811,-9.372225
2,"Middle East, North Africa, Afghanistan & Pakistan",89.024141,101.441811,12.417670
3,South Asia,108.019121,101.441811,-6.577310
4,Sub-Saharan Africa,92.281592,101.441811,9.160219


In [50]:
pd.read_sql('''
WITH regional_secondary AS (
    SELECT
        region,
        AVG(value) AS avg_secondary_rate
    FROM wdi_edu
    WHERE series = 'secondary'
      AND region NOT IN ('World', 'Europe & Central Asia', 'North America')
    GROUP BY region
),
benchmark AS (
    SELECT
        AVG(value) AS benchmark_rate
    FROM wdi_edu
    WHERE series = 'secondary'
      AND region IN ('Europe & Central Asia', 'North America')
)
SELECT
    r.region,
    r.avg_secondary_rate,
    b.benchmark_rate,
    (b.benchmark_rate - r.avg_secondary_rate) AS gap_from_benchmark
FROM regional_secondary r
CROSS JOIN benchmark b;
''', conn)


,region,avg_secondary_rate,benchmark_rate,gap_from_benchmark
0,East Asia & Pacific,79.326682,98.696142,19.369460
1,Latin America & Caribbean,89.990020,98.696142,8.706122
2,"Middle East, North Africa, Afghanistan & Pakistan",55.052961,98.696142,43.643181
3,South Asia,62.982683,98.696142,35.713458
4,Sub-Saharan Africa,36.812869,98.696142,61.883273


In [51]:
pd.read_sql('''
WITH regional_tertiary AS (
    SELECT
        region,
        AVG(value) AS avg_tertiary_rate
    FROM wdi_edu
    WHERE series = 'tertiary'
      AND region NOT IN ('World', 'Europe & Central Asia', 'North America')
    GROUP BY region
),
benchmark AS (
    SELECT
        AVG(value) AS benchmark_rate
    FROM wdi_edu
    WHERE series = 'tertiary'
      AND region IN ('Europe & Central Asia', 'North America')
)
SELECT
    r.region,
    r.avg_tertiary_rate,
    b.benchmark_rate,
    (b.benchmark_rate - r.avg_tertiary_rate) AS gap_from_benchmark
FROM regional_tertiary r
CROSS JOIN benchmark b;
''', conn)


,region,avg_tertiary_rate,benchmark_rate,gap_from_benchmark
0,East Asia & Pacific,32.326086,72.023731,39.697645
1,Latin America & Caribbean,40.691914,72.023731,31.331817
2,"Middle East, North Africa, Afghanistan & Pakistan",21.230170,72.023731,50.793561
3,South Asia,18.264229,72.023731,53.759503
4,Sub-Saharan Africa,7.289573,72.023731,64.734158


# Average Gender Gaps in Each Education Level (World)

In [ ]:
pd.read_sql('''
WITH male AS (
    SELECT region, year, series, value AS male_rate
    FROM wdi_edu
    WHERE series LIKE '%_male'
),
female AS (
    SELECT region, year, series, value AS female_rate
    FROM wdi_edu
    WHERE series LIKE '%_female'
),
paired AS (
    SELECT
        m.region,
        m.year,
        REPLACE(m.series, '_male', '') AS level,
        m.male_rate,
        f.female_rate,
        m.male_rate - f.female_rate AS gender_gap
    FROM male m
    JOIN female f
      ON m.region = f.region
     AND m.year = f.year
     AND REPLACE(m.series, '_male', '') = REPLACE(f.series, '_female', '')
)
SELECT
    level,
    AVG(male_rate)   AS avg_male_rate,
    AVG(female_rate) AS avg_female_rate,
    AVG(gender_gap)  AS avg_gender_gap
FROM paired
GROUP BY level
ORDER BY level;
''', conn)

,level,avg_male_rate,avg_female_rate,avg_gender_gap
0,primary,102.625246,99.523326,3.101920
1,secondary,74.752978,72.962301,1.790677
2,tertiary,33.892155,39.691699,-5.799544


In [57]:
pd.read_sql('''
WITH male AS (
    SELECT region, year, series, value AS male_rate
    FROM wdi_edu
    WHERE series LIKE '%_male'
),
female AS (
    SELECT region, year, series, value AS female_rate
    FROM wdi_edu
    WHERE series LIKE '%_female'
),
paired AS (
    SELECT
        m.region,
        REPLACE(m.series, '_male', '') AS level,
        m.male_rate,
        f.female_rate,
        m.male_rate - f.female_rate AS gender_gap
    FROM male m
    JOIN female f
      ON m.region = f.region
     AND m.year = f.year
     AND REPLACE(m.series, '_male', '') = REPLACE(f.series, '_female', '')
)
SELECT
    region,
    level,
    AVG(male_rate)   AS avg_male_rate,
    AVG(female_rate) AS avg_female_rate,
    AVG(gender_gap)  AS avg_gender_gap
FROM paired
GROUP BY region, level
ORDER BY region, level;
''', conn)


,region,level,avg_male_rate,avg_female_rate,avg_gender_gap
0,East Asia & Pacific,primary,104.438934,104.644204,-0.205270
1,East Asia & Pacific,secondary,78.887388,79.819653,-0.932265
2,East Asia & Pacific,tertiary,31.243773,33.534001,-2.290228
3,Europe & Central Asia,primary,102.400096,101.555958,0.844138
4,Europe & Central Asia,secondary,100.954070,99.471357,1.482713
5,Europe & Central Asia,tertiary,56.636315,66.740904,-10.104589
6,Latin America & Caribbean,primary,112.137270,109.438046,2.699223
7,Latin America & Caribbean,secondary,87.217975,92.848324,-5.630349
8,Latin America & Caribbean,tertiary,35.859428,45.605031,-9.745603
9,"Middle East, North Africa, Afghanistan & Pakistan",primary,94.402242,83.330292,11.071950
